In [1]:
import benepar
from IPython.core.debugger import set_trace
import numpy as np
import os

parser = benepar.Parser("../brain_syntactic_representations/benepar_en2_large/")

save_dir = "features"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [2]:
all_text = []
hp_text = []
para = ""

In [3]:
narrative_text = open("./stimuli/text/lw1new.txt","r").readlines()
for eachsent in narrative_text[0].split('.'):
    all_text.append(eachsent.strip()+'.')
    for j in range(len(eachsent.strip().split())):
        #if eachsent.strip().split()[j].strip() not in [',','?','!','-','"',';']:
        hp_text.append(eachsent.strip().split()[j].strip())
print(all_text)

narrative_text = open("./stimuli/text/cable_spool_fort_new.txt","r").readlines()
for eachsent in narrative_text[0].split('.'):
    all_text.append(eachsent.strip()+'.')
    for j in range(len(eachsent.strip().split())):
        #if eachsent.strip().split()[j].strip() not in [',','?','!','-','"',';']:
        hp_text.append(eachsent.strip().split()[j].strip())
        
narrative_text = open("./stimuli/text/easy_money_new.txt","r").readlines()
for eachsent in narrative_text[0].split('.'):
    all_text.append(eachsent.strip()+'.')
    for j in range(len(eachsent.strip().split())):
        #if eachsent.strip().split()[j].strip() not in [',','?','!','-','"',';']:
        hp_text.append(eachsent.strip().split()[j].strip())
        
narrative_text = open("./stimuli/text/black_willow_new.txt","r").readlines()
for eachsent in narrative_text[0].split('.'):
    all_text.append(eachsent.strip()+'.')
    for j in range(len(eachsent.strip().split())):
        #if eachsent.strip().split()[j].strip() not in [',','?','!','-','"',';']:
        hp_text.append(eachsent.strip().split()[j].strip())

['Tara stood stock still waiting for the first tiny gleam from the scout craft to appear in the darkness of the.', 'The gentle constant breeze of recycled air from the vent above blew an annoying hair against her nose but she ignored it.', 'A gasp from the psychic broke her silent vigil and she turned.', 'Results Harmon she suppressed the surge of annoyance that ran through her as she contemplated the gift of getting all the hot news first.', 'face slowly animated joy sweeping in to replace stern concentration.', 'says the planet tiny away contemplated to not blonde slowly broke by Thriving with life large and small forms no buildings of any kind metals and stable atmosphere excellent onto the only head planet pointed blackness now get gentle imposing alpha.', 'He says we should come now immediately before the probe is reported late Harmon that good nothing son of beta get his the gate and here.', 'NOW or open fire on him when we come Tara Of all the no doubt with orders from Mason was

In [4]:
all_subtrees = []
for i in range(len(hp_text)):
    all_subtrees.append([])
    
all_lexical_prods = []
    
ptr = 0
for para in all_text:
    trees = parser.parse_sents(para)
    for tree in trees:
        productions = tree.productions()
        cur_prods = []
        len_remaining = len(hp_text[ptr])
        for prod in productions:
            cur_prods.append(prod)
            if prod.is_lexical():
                if prod.rhs()[0] == "``" or prod.rhs()[0] == "''" or prod.rhs()[0] == "-LRB-" or prod.rhs()[0] == "-RRB-":
                        len_remaining -= 1
                else:
                    len_remaining -= len(prod.rhs()[0])
                all_lexical_prods.append(prod)
            if len_remaining == 0:
                all_subtrees[ptr] = cur_prods.copy()
                ptr += 1
                if hp_text[ptr] == "+":
                    ptr += 1
                if ptr<len(hp_text) and hp_text[ptr] == "+":
                    ptr += 1
                if ptr<len(hp_text):
                    len_remaining = len(hp_text[ptr])

In [5]:
import pickle
with open("all_subtrees_nodecnt.pkl","wb") as f:
    pickle.dump(all_subtrees,f)

In [6]:
with open("all_subtrees_nodecnt.pkl","rb") as f:
    bck = pickle.load(f)
all_end_subtrees = []
for i in range(len(hp_text)):
    all_end_subtrees.append([])

node_cnts = []
for i in range(len(hp_text)):
    node_cnts.append(0)

for i in range(len(hp_text)):
    j = len(all_subtrees[i]) - 1
    prev = []
    while(j>=0):
        cur = bck[i][j]
        rhs = cur.rhs()
        if cur.is_lexical() and len(cur._rhs)>0:
            all_end_subtrees[i] = [all_subtrees[i][j]] + all_end_subtrees[i]
            cur._rhs = ()
            j -= 1
            prev.append(cur.lhs())
            node_cnts[i] += 1
        else:
            checkrem = 0
            rem = 0
            for sym in rhs:
                if sym in prev:
                    rem += 1
                    prev.remove(sym)
                    checkrem = 1
                if len(prev) == 0:
                    break
            bck[i][j]._rhs = bck[i][j]._rhs[rem:]
            if checkrem == 1 and len(bck[i][j]._rhs) == 0:
                node_cnts[i] += 1
                all_end_subtrees[i] = all_subtrees[i][j:]
                prev.append(all_subtrees[i][j].lhs())
            j -= 1

In [7]:
for i in range(len(all_end_subtrees)):
    print(hp_text[i])
    print(node_cnts[i])
    print(all_subtrees[i])
    print(all_end_subtrees[i])

Tara
2
[S -> NP VP ., NP -> NNP, NNP -> 'Tara']
[NP -> NNP, NNP -> 'Tara']
stood
1
[S -> NP VP ., NP -> NNP, NNP -> 'Tara', VP -> VBD NP S, VBD -> 'stood']
[VBD -> 'stood']
stock
2
[S -> NP VP ., NP -> NNP, NNP -> 'Tara', VP -> VBD NP S, VBD -> 'stood', NP -> NN, NN -> 'stock']
[NP -> NN, NN -> 'stock']
still
2
[S -> NP VP ., NP -> NNP, NNP -> 'Tara', VP -> VBD NP S, VBD -> 'stood', NP -> NN, NN -> 'stock', S -> VP, VP -> ADVP S, ADVP -> RB, RB -> 'still']
[ADVP -> RB, RB -> 'still']
waiting
1
[S -> NP VP ., NP -> NNP, NNP -> 'Tara', VP -> VBD NP S, VBD -> 'stood', NP -> NN, NN -> 'stock', S -> VP, VP -> ADVP S, ADVP -> RB, RB -> 'still', S -> VP, VP -> VBG SBAR, VBG -> 'waiting']
[VBG -> 'waiting']
for
1
[S -> NP VP ., NP -> NNP, NNP -> 'Tara', VP -> VBD NP S, VBD -> 'stood', NP -> NN, NN -> 'stock', S -> VP, VP -> ADVP S, ADVP -> RB, RB -> 'still', S -> VP, VP -> VBG SBAR, VBG -> 'waiting', SBAR -> IN S, IN -> 'for']
[IN -> 'for']
the
1
[S -> NP VP ., NP -> NNP, NNP -> 'Tara', VP -> 

[]
one
0
[]
[]
from
0
[]
[]
a
0
[]
[]
truck
0
[]
[]
and
0
[]
[]
two
0
[]
[]
smaller
0
[]
[]
ones
0
[]
[]
from
0
[]
[]
cars
0
[]
[]
The
0
[]
[]
older
0
[]
[]
boys
0
[]
[]
lay
0
[]
[]
around
0
[]
[]
in
0
[]
[]
shade
0
[]
[]
and
0
[]
[]
napped
0
[]
[]
inside
0
[]
[]
the
0
[]
[]
walls
0
[]
[]
thing
0
[]
[]
could
0
[]
[]
while
0
[]
[]
than
0
[]
[]
blinked
0
[]
[]
breath
0
[]
[]
corner
0
[]
[]
hard
0
[]
[]
cinder
0
[]
[]
boy
0
[]
[]
force
0
[]
[]
this
0
[]
[]
put
0
[]
[]
faster
0
[]
[]
weapons
0
[]
[]
sob
0
[]
[]
brain
0
[]
[]
start
0
[]
[]
They
0
[]
[]
were
0
[]
[]
watching
0
[]
[]
now
0
[]
[]
from
0
[]
[]
well
0
[]
[]
defended
0
[]
[]
spy
0
[]
[]
holes
0
[]
[]
or
0
[]
[]
maybe
0
[]
[]
they
0
[]
[]
were
0
[]
[]
performing
0
[]
[]
secret
0
[]
[]
experiments
0
[]
[]
and
0
[]
[]
swearing
0
[]
[]
bloody
0
[]
[]
oaths
0
[]
[]
Chad
0
[]
[]
wished
0
[]
[]
he
0
[]
[]
were
0
[]
[]
old
0
[]
[]
enough
0
[]
[]
to
0
[]
[]
be
0
[]
[]
one
0
[]
[]
of
0
[]
[]
them
0
[]
[]
Maybe
0
[]
[]
Tucker
0
[]
[]
would


[]
[]
pulling
0
[]
[]
his
0
[]
[]
eyes
0
[]
[]
wider
0
[]
[]
still
0
[]
[]
From
0
[]
[]
his
0
[]
[]
pocket
0
[]
[]
pulled
0
[]
[]
a
0
[]
[]
small
0
[]
[]
screwdriver
0
[]
[]
perfect
0
[]
[]
for
0
[]
[]
prying
0
[]
[]
plugs
0
[]
[]
from
0
[]
[]
heads
0
[]
[]
Now
0
[]
[]
legs
0
[]
[]
and
0
[]
[]
arms
0
[]
[]
and
0
[]
[]
mouth
0
[]
[]
worked
0
[]
[]
and
0
[]
[]
was
0
[]
[]
up
0
[]
[]
and
0
[]
[]
running
0
[]
[]
and
0
[]
[]
screaming
0
[]
[]
Roy
0
[]
[]
go
0
[]
[]
inside
0
[]
[]
remote
0
[]
[]
room
0
[]
[]
voice
0
[]
[]
truck
0
[]
[]
would
0
[]
[]
so
0
[]
[]
what
0
[]
[]
going
0
[]
[]
silver
0
[]
[]
taught
0
[]
[]
screaming
0
[]
[]
toads
0
[]
[]
play
0
[]
[]
being
0
[]
[]
Roy
0
[]
[]
He
0
[]
[]
sprinted
0
[]
[]
for
0
[]
[]
the
0
[]
[]
corner
0
[]
[]
of
0
[]
[]
the
0
[]
[]
building
0
[]
[]
around
0
[]
[]
which
0
[]
[]
Roy
0
[]
[]
had
0
[]
[]
disappeared
0
[]
[]
The
0
[]
[]
door
0
[]
[]
was
0
[]
[]
on
0
[]
[]
that
0
[]
[]
side
0
[]
[]
and
0
[]
[]
Rayburn
0
[]
[]
Just
0
[]
[]
then
0
[]
[]
a
0

[]
[]
my
0
[]
[]
life
0
[]
[]
taking
0
[]
[]
kind
0
[]
[]
chance
0
[]
[]
I
0
[]
[]
put
0
[]
[]
away
0
[]
[]
the
0
[]
[]
cell
0
[]
[]
and
0
[]
[]
used
0
[]
[]
the
0
[]
[]
motion
0
[]
[]
to
0
[]
[]
cover
0
[]
[]
checking
0
[]
[]
the
0
[]
[]
knife
0
[]
[]
in
0
[]
[]
my
0
[]
[]
sleeve
0
[]
[]
and
0
[]
[]
used
0
[]
[]
one
0
[]
[]
leg
0
[]
[]
to
0
[]
[]
check
0
[]
[]
the
0
[]
[]
other
0
[]
[]
in
0
[]
[]
my
0
[]
[]
sock
0
[]
[]
I
0
[]
[]
had
0
[]
[]
been
0
[]
[]
an
0
[]
[]
amateur
0
[]
[]
magician
0
[]
[]
for
0
[]
[]
years
0
[]
[]
One
0
[]
[]
of
0
[]
[]
my
0
[]
[]
best
0
[]
[]
tricks
0
[]
[]
was
0
[]
[]
pulling
0
[]
[]
a
0
[]
[]
knife
0
[]
[]
out
0
[]
[]
of
0
[]
[]
thin
0
[]
[]
air
0
[]
[]
It
0
[]
[]
never
0
[]
[]
failed
0
[]
[]
to
0
[]
[]
amaze
0
[]
[]
Sure
0
[]
[]
go
0
[]
[]
I
0
[]
[]
said
0
[]
[]
We
0
[]
[]
took
0
[]
[]
his
0
[]
[]
rental
0
[]
[]
car
0
[]
[]
back
0
[]
[]
to
0
[]
[]
the
0
[]
[]
motel
0
[]
[]
where
0
[]
[]
he
0
[]
[]
was
0
[]
[]
staying
0
[]
[]
The
0
[]
[]
little
0
[]
[]
des

0
[]
[]
said
0
[]
[]
I
0
[]
[]
I
0
[]
[]
I
0
[]
[]
expected
0
[]
[]
him
0
[]
[]
pull
0
[]
[]
out
0
[]
[]
a
0
[]
[]
gadget
0
[]
[]
blinking
0
[]
[]
lights
0
[]
[]
and
0
[]
[]
was
0
[]
[]
wondering
0
[]
[]
which
0
[]
[]
way
0
[]
[]
duck
0
[]
[]
Trick
0
[]
[]
gesture
0
[]
[]
like
0
[]
[]
sleight
0
[]
[]
of
0
[]
[]
hand
0
[]
[]
He
0
[]
[]
his
0
[]
[]
fingers
0
[]
[]
and
0
[]
[]
then
0
[]
[]
blew
0
[]
[]
on
0
[]
[]
them
0
[]
[]
OK
0
[]
[]
watch
0
[]
[]
and
0
[]
[]
remember
0
[]
[]
this
0
[]
[]
going
0
[]
[]
to
0
[]
[]
do
0
[]
[]
it
0
[]
[]
in
0
[]
[]
two
0
[]
[]
parts
0
[]
[]
last
0
[]
[]
part
0
[]
[]
first
0
[]
[]
so
0
[]
[]
it
0
[]
[]
take
0
[]
[]
effect
0
[]
[]
Remember
0
[]
[]
to
0
[]
[]
reverse
0
[]
[]
the
0
[]
[]
two
0
[]
[]
parts
0
[]
[]
when
0
[]
[]
you
0
[]
[]
actually
0
[]
[]
do
0
[]
[]
it
0
[]
[]
Stunned
0
[]
[]
I
0
[]
[]
watched
0
[]
[]
as
0
[]
[]
he
0
[]
[]
made
0
[]
[]
several
0
[]
[]
ridiculous
0
[]
[]
passes
0
[]
[]
with
0
[]
[]
his
0
[]
[]
hands
0
[]
[]
paused
0
[]
[]
and
0

...

[]
Easter
0
[]
[]
Island
0
[]
[]
even
0
[]
[]
cable
0
[]
[]
prosecution
0
[]
[]
he
0
[]
[]
ounces
0
[]
[]
tone
0
[]
[]
blinking
0
[]
[]
hand
0
[]
[]
ill
0
[]
[]
people
0
[]
[]
see
0
[]
[]
hill
0
[]
[]
I
0
[]
[]
figure
0
[]
[]
be
0
[]
[]
worse
0
[]
[]
than
0
[]
[]
the
0
[]
[]
lethal
0
[]
[]
injection
0
[]
[]
waiting
0
[]
[]
for
0
[]
[]
me
0
[]
[]
at
0
[]
[]
dawn
0
[]
[]
I
0
[]
[]
do
0
[]
[]
matter
0
[]
[]
tell
0
[]
[]
you
0
[]
[]
because
0
[]
[]
there
0
[]
[]
be
0
[]
[]
jail
0
[]
[]
can
0
[]
[]
hold
0
[]
[]
me
0
[]
[]
leaving
0
[]
[]
this
0
[]
[]
tell
0
[]
[]
whom
0
[]
[]
may
0
[]
[]
concern
0
[]
[]
kiss
0
[]
[]
ever
0
[]
[]
ass
0
[]
[]
Wish
0
[]
[]
me
0
[]
[]
luck
0
[]
[]
out
0
[]
[]
of
0
[]
[]
The
0
[]
[]
Black
0
[]
[]
Willow
0
[]
[]
Allan
0
[]
[]
sat
0
[]
[]
down
0
[]
[]
at
0
[]
[]
his
0
[]
[]
desk
0
[]
[]
and
0
[]
[]
pulled
0
[]
[]
the
0
[]
[]
chair
0
[]
[]
in
0
[]
[]
close
0
[]
[]
legs
0
[]
[]
love
0
[]
[]
forced
0
[]
[]
windows
0
[]
[]
leaps
0
[]
[]
imagination
0
[]
[]
overshadowe

had
0
[]
[]
a
0
[]
[]
perfect
0
[]
[]
story
0
[]
[]
in
0
[]
[]
my
0
[]
[]
pages
0
[]
[]
my
0
[]
[]
hero
0
[]
[]
returned
0
[]
[]
home
0
[]
[]
and
0
[]
[]
reclaimed
0
[]
[]
his
0
[]
[]
love
0
[]
[]
I
0
[]
[]
nearly
0
[]
[]
had
0
[]
[]
it
0
[]
[]
tied
0
[]
[]
together
0
[]
[]
before
0
[]
[]
Before
0
[]
[]
Before
0
[]
[]
that
0
[]
[]
blasted
0
[]
[]
impulse
0
[]
[]
came
0
[]
[]
over
0
[]
[]
me
0
[]
[]
and
0
[]
[]
I
0
[]
[]
started
0
[]
[]
writing
0
[]
[]
like
0
[]
[]
Edgar
0
[]
[]
Poe
0
[]
[]
though
0
[]
[]
the
0
[]
[]
most
0
[]
[]
recent
0
[]
[]
was
0
[]
[]
not
0
[]
[]
so
0
[]
[]
morbid
0
[]
[]
as
0
[]
[]
my
0
[]
[]
many
0
[]
[]
others
0
[]
[]
thank
0
[]
[]
Heaven
0
[]
[]
face
0
[]
[]
assumed
0
[]
[]
a
0
[]
[]
long
0
[]
[]
suffering
0
[]
[]
aspect
0
[]
[]
of
0
[]
[]
authoritative
0
[]
[]
I
0
[]
[]
assume
0
[]
[]
referring
0
[]
[]
to
0
[]
[]
your
0
[]
[]
unhealthy
0
[]
[]
for
0
[]
[]
the
0
[]
[]
fantastic
0
[]
[]
far
0
[]
[]
from
0
[]
[]
a
0
[]
[]
Arthur
0
[]
[]
This
0
[]
[]
is
0
[]
[]
mo

[]
our
0
[]
[]
real
0
[]
[]
world
0
[]
[]
characters
0
[]
[]
who
0
[]
[]
might
0
[]
[]
live
0
[]
[]
therein
0
[]
[]
not
0
[]
[]
citizens
0
[]
[]
some
0
[]
[]
airy
0
[]
[]
beyond
0
[]
[]
the
0
[]
[]
stars
0
[]
[]
Why
0
[]
[]
is
0
[]
[]
that
0
[]
[]
more
0
[]
[]
desirable
0
[]
[]
than
0
[]
[]
what
0
[]
[]
you
0
[]
[]
write
0
[]
[]
today
0
[]
[]
People
0
[]
[]
will
0
[]
[]
such
0
[]
[]
stories
0
[]
[]
they
0
[]
[]
are
0
[]
[]
worth
0
[]
[]
publishing
0
[]
[]
If
0
[]
[]
you
0
[]
[]
never
0
[]
[]
finish
0
[]
[]
such
0
[]
[]
stories
0
[]
[]
how
0
[]
[]
can
0
[]
[]
they
0
[]
[]
be
0
[]
[]
worth
0
[]
[]
publishing
0
[]
[]
That
0
[]
[]
is
0
[]
[]
precisely
0
[]
[]
why
0
[]
[]
I
0
[]
[]
must
0
[]
[]
excise
0
[]
[]
this
0
[]
[]
malignant
0
[]
[]
instinct
0
[]
[]
Why
0
[]
[]
do
0
[]
[]
you
0
[]
[]
write
0
[]
[]
Why
0
[]
[]
to
0
[]
[]
earn
0
[]
[]
a
0
[]
[]
living
0
[]
[]
I
0
[]
[]
suppose
0
[]
[]
Why
0
[]
[]
do
0
[]
[]
you
0
[]
[]
ask
0
[]
[]
You
0
[]
[]
choose
0
[]
[]
to
0
[]
[]
write
0
[]
[]
rat

[]
without
0
[]
[]
assistance
0
[]
[]
and
0
[]
[]
setting
0
[]
[]
off
0
[]
[]
on
0
[]
[]
foot
0
[]
[]
towards
0
[]
[]
Nathan
0
[]
[]
read
0
[]
[]
the
0
[]
[]
package
0
[]
[]
of
0
[]
[]
words
0
[]
[]
in
0
[]
[]
silence
0
[]
[]
his
0
[]
[]
only
0
[]
[]
motions
0
[]
[]
the
0
[]
[]
steady
0
[]
[]
progress
0
[]
[]
of
0
[]
[]
his
0
[]
[]
eyes
0
[]
[]
and
0
[]
[]
occasional
0
[]
[]
replacement
0
[]
[]
of
0
[]
[]
pages
0
[]
[]
Allan
0
[]
[]
sat
0
[]
[]
across
0
[]
[]
from
0
[]
[]
him
0
[]
[]
in
0
[]
[]
a
0
[]
[]
chair
0
[]
[]
Nathan
0
[]
[]
had
0
[]
[]
probably
0
[]
[]
upholstered
0
[]
[]
himself
0
[]
[]
a
0
[]
[]
patchwork
0
[]
[]
design
0
[]
[]
of
0
[]
[]
fabric
0
[]
[]
containing
0
[]
[]
easily
0
[]
[]
more
0
[]
[]
stuffing
0
[]
[]
than
0
[]
[]
any
0
[]
[]
other
0
[]
[]
furniture
0
[]
[]
item
0
[]
[]
of
0
[]
[]
the
0
[]
[]
period
0
[]
[]
At
0
[]
[]
long
0
[]
[]
last
0
[]
[]
Nathan
0
[]
[]
reached
0
[]
[]
the
0
[]
[]
end
0
[]
[]
and
0
[]
[]
set
0
[]
[]
down
0
[]
[]
his
0
[]
[]
reading
0
[]
[

[]
[]
smooth
0
[]
[]
bark
0
[]
[]
a
0
[]
[]
gray
0
[]
[]
green
0
[]
[]
pennant
0
[]
[]
on
0
[]
[]
a
0
[]
[]
slope
0
[]
[]
and
0
[]
[]
standing
0
[]
[]
out
0
[]
[]
between
0
[]
[]
the
0
[]
[]
dark
0
[]
[]
cracked
0
[]
[]
trunks
0
[]
[]
of
0
[]
[]
the
0
[]
[]
nearer
0
[]
[]
trees
0
[]
[]
great
0
[]
[]
pines
0
[]
[]
a
0
[]
[]
few
0
[]
[]
magnificent
0
[]
[]
specimens
0
[]
[]
still
0
[]
[]
bearing
0
[]
[]
the
0
[]
[]
three
0
[]
[]
stroke
0
[]
[]
symbol
0
[]
[]
of
0
[]
[]
the
0
[]
[]
British
0
[]
[]
King
0
[]
[]
a
0
[]
[]
towering
0
[]
[]
black
0
[]
[]
birch
0
[]
[]
leaves
0
[]
[]
impossibly
0
[]
[]
high
0
[]
[]
above
0
[]
[]
him
0
[]
[]
As
0
[]
[]
the
0
[]
[]
stream
0
[]
[]
wound
0
[]
[]
its
0
[]
[]
way
0
[]
[]
through
0
[]
[]
the
0
[]
[]
largest
0
[]
[]
pines
0
[]
[]
grown
0
[]
[]
so
0
[]
[]
great
0
[]
[]
as
0
[]
[]
to
0
[]
[]
eclipse
0
[]
[]
that
0
[]
[]
feature
0
[]
[]
of
0
[]
[]
the
0
[]
[]
landscape
0
[]
[]
they
0
[]
[]
had
0
[]
[]
once
0
[]
[]
crowded
0
[]
[]
about
0
[]
[]
for
0
[]
[

In [8]:
np.save(os.path.join(save_dir,"node_count"),node_cnts)

In [9]:
#stories = ['lw1','cable','easymoney','balckwillow']
story_len = [0,668,1503,2637,3753]
#final = np.load("./features/pos_tags_all.npy", allow_pickle=True)
nodecount = []
for i in np.arange(1,5):
    #aa = np.load("./features/pos_tags_"+eachstory+".npy", allow_pickle=True)
    nodecount.append(node_cnts[story_len[i-1]:story_len[i-1]+story_len[i]])

In [10]:
np.save('nodecount_allstories', nodecount)

/home/subbainria/anaconda3/lib/python3.8/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
